### 1. 介绍

- **基础知识**：LPP 相比于 LE，新增了投影问题，设$X,Y,V$ 都是按列排列的，即每一列是一个样本，则投影关系为： $$Y = V^T X$$

- **降维方式**：线性降维

- **重点**：先按LEE算出初步矩阵$W_{i,j}$，然后用“图G逆”的方式计算$W$，然后按LPP方法求$V$，最后取<mark>从第2小到第(d+1)小<mark>特征值对应的特征向量。
  - **G逆的方式**：
  设 $G$ 为与点 $i$ 相关的局部 Grammian 矩阵，其条目定义为：
  $$
  g_{pl} = (x_i - x_p)^\top (x_i - x_l) \in \mathbb{R}^{k \times k}。
  $$
  因此，$G$ 包含了 $x_i$ 的邻居之间的成对内积，假定邻居相对于 $x_i$ 进行了中心化。用 $X^{(i)}$ 表示由 $x_i$ 及其邻居组成的向量系统，我们需要解决最小二乘问题 $(X^{(i)} - x_i e^{\top}) w_{i,:} = 0$，并满足约束 $e^\top w_{i,:} = 1$。可以证明，该约束最小二乘问题的解 $w_{i,:}$ 如下所示 [1]，涉及 $G$ 的逆：
  $$
  w_{i,:} = \frac{G^{-1} e}{e^\top G^{-1} e}。
  $$


- **OLPP**：正交邻域保持投影（Orthogonal Neighborhood Preserving Projections，ONPP）
  - **描述**：ONPP算法先根据最近邻亲和图来确定$W$，然后按照LPP的流程降维。
  - 相同之处：类似于LLE的方法构建数据相似度的图$G$
  - 不同之处：LLE中的输入空间和降维空间之间的映射是隐式的，而ONPP则使用显式的线性映射。

- **归纳目标函数**：
  - **LPP**：$$X(D-W)X^\top V=\Lambda XDX^\top V$$
  - **OLPP**：对于由于$d_i=1$，所以$D$变成$I$，而$W$改为由$$w_{i,:}=\frac{G^{-1}e}{e^{\top}G^{-1}e},\quad  g_{pl} = (x_i - x_p)^\top (x_i - x_l) \in \mathbb{R}^{k \times k}$$确定，然后按LPP的方法计算$V$,<mark>取从第2小到第(d+1)小<mark>特征值对应的特征向量（因为最后一个特征值是0）

### 2. 流程

#### 2.2. 大致步骤

**算法：$\mathbf{ONPP}$**

**输入**: 数据集 $X \in \mathbb{R}^{m \times n}$ 和降维后的空间维度 $d$。

**输出**: 嵌入向量 $Y \in \mathbb{R}^{d \times n}$。

1. 计算每个数据点的 $k$ 个最近邻。
2. 计算权重 $w_{ij}$
3. 计算矩阵 $V$，其列向量是矩阵
   $$
   \tilde{M} = X (I - W^\top) (I - W) X^\top
   $$
   的<mark>从第2小到第(d+1)小<mark>特征值对应的特征向量。
4. 计算投影向量 $y_i = V^\top x_i$。
   
#### 2.2. 详细流程

1. **计算最近邻**:
   - 对于数据集中的每个数据点 $x_i$，使用欧氏距离或其他适当的距离度量找到其 $k$ 个最近邻点。
2. **计算重构权重**:
   - 对于每个数据点 $x_i$，构建其近邻矩阵 $X^{(i)}$。
   - 通过最小化以下重构误差计算权重：
     $$
     \mathcal{E}(W) = \sum_i \|x_i - \sum_j w_{ij} x_j\|_2^2
     $$
   - 权重的计算满足以下约束：
     $$
     \sum_j w_{ij} = 1 \quad \text{且} \quad w_{ij} = 0 \text{，如果 } x_j \text{ 不是 } x_i \text{ 的 } k \text{ 近邻之一}
     $$
   - 使用局部Grammian矩阵 $G$ 和所有元素均为1的向量 $e$ 计算权重：
     $$
     w_{i,:} = \frac{G^{-1} e}{e^\top G^{-1} e}
     $$
3. **计算矩阵 $V$**:
   - 构建矩阵：
     $$
     \tilde{M} = X (I - W^\top) (I - W) X^\top
     $$
   - 求解 $\tilde{M}$ 的特征值问题，找到从第2小到第 $(d+1)$ 小的特征值对应的特征向量，组成矩阵 $V$。
4. **计算投影向量**:
   - 通过矩阵 $V$ 和原始数据 $x_i$ 计算投影向量：
     $$
     y_i = V^\top x_i
     $$

### 3.摘译

#### 3.1. 摘要

本文研究了通过正交投影技术进行降维的问题。所提出技术的主要特点是尝试同时保留数据样本的内在邻域几何结构和全局几何结构。具体而言，我们提出了一种方法，称为正交邻域保持投影（Orthogonal Neighborhood Preserving Projections，ONPP），该方法首先以类似于局部线性嵌入（Locally Linear Embedding，LLE）的方法构建数据的“亲和”图。然而，与标准的LLE不同的是，LLE中的输入空间和降维空间之间的映射是隐式的，而ONPP则使用显式的线性映射。因此，处理新数据样本变得非常简单，因为这仅相当于一个简单的线性变换。我们展示了如何定义ONPP的核变体，以及如何在监督设置中应用该方法。报告的数值实验展示了ONPP的性能，并将其与一些竞争方法进行了比较。

#### 3.2. 最近邻亲和图
考虑一个由矩阵 $X=[x_{1},x_{2},\ldots,x_{n}] \in \mathbb{R}^{m \times n}$ 的列表示的数据集。ONPP 首先通过计算最佳权重来构建亲和矩阵，这些权重将以某种局部最优的方式将一个给定点与其邻居联系起来。这个阶段与 LLE 的方法相同 [1], [2]。
为了完整起见，这里总结了构建亲和图的过程，详细信息可以在 [1] 和 [2] 中找到。基本假设是每个数据样本及其 $k$ 近邻（近似）位于一个局部线性流形上。因此，每个数据样本 $x_i$ 由其 $k$ 近邻的线性组合重构。通过最小化目标函数来衡量重构误差：
$$
\mathcal{E}(W) = \sum_i \|x_i - \sum_j w_{ij} x_j\|_2^2。
$$
权重 $w_{ij}$ 表示从其邻居 $\{x_j\}$ 重构样本 $x_i$ 的线性系数。对权重施加以下约束：
$$
\begin{array}{ll}
1. & w_{ij} = 0, \text{如果 } x_j \text{ 不是 } x_i \text{ 的 } k\text{-近邻之一。}\\
2. & \sum_{j} w_{ij} = 1, \text{即 } x_i \text{ 是其邻居的凸组合的近似值。}
\end{array}
$$
注意，行和上的第二个约束类似于令 $ d_i = 1$，其中 $d_i$ 定义在 (3) 中。因此，施加此约束相当于在第 2 节中重新缩放矩阵 $W$ 以使其产生等于单位矩阵的 $D$ 矩阵。
在所有 $i$ 都满足 $w_{ii} \equiv 0$ 的情况下，该问题等价于找到一个具有指定稀疏模式的稀疏矩阵 $Z'$（$Z \equiv I - W^{\top}$），其对角线上的元素为 1，且其行和全为零。
权重的计算有一个简单的闭式表达式。值得指出的是，对于给定点 $x_i$，确定 $w_{ij}$ 是一个局部计算，因为它仅涉及 $x_i$ 及其近邻。任何用于计算权重的算法都将相对便宜。
设 $G$ 为与点 $i$ 相关的局部 Grammian 矩阵，其条目定义为：
$$
g_{pl} = (x_i - x_p)^\top (x_i - x_l) \in \mathbb{R}^{k \times k}。
$$
因此，$G$ 包含了 $x_i$ 的邻居之间的成对内积，假定邻居相对于 $x_i$ 进行了中心化。用 $X^{(i)}$ 表示由 $x_i$ 及其邻居组成的向量系统，我们需要解决最小二乘问题 $(X^{(i)} - x_i e^{\top}) w_{i,:} = 0$，并满足约束 $e^\top w_{i,:} = 1$。可以证明，该约束最小二乘问题的解 $w_{i,:}$ 如下所示 [1]，涉及 $G$ 的逆：
$$
w_{i,:} = \frac{G^{-1} e}{e^\top G^{-1} e}。
$$
请记住，$e$ 是所有元素均为 1 的向量。权重 $w_{ij}$ 满足某些最优性质。它们对旋转、各向同性缩放和平移不变。由于这些性质，亲和图保留了每个邻域的内在几何特性。